# Advanced Collections: _Custom Operations_

In the overview notabook we discussed some of the many algorithms that are pre-defined for different types of Dask collections
(such as Arrays and DataFrames). These include operations like `mean`, `max`, `value_counts` and many other standard operations.

In this notebook we'll explore how those operations are implemented and learn how  to construct our own custom operations to use with Dask Arrays and Dask Dataframes.


**Related Documentation**

  - [Array Tutorial](https://tutorial.dask.org/03_array.html)
  - [Best Practices](https://docs.dask.org/en/latest/best-practices.html#learn-techniques-for-customization)

## Blocked Algorithms

Dask computations are implemented using _blocked algorithms_. These algorithms break up a computation on a large array into many computations on smaller pieces of the array. This minimizes the memory load (amount of RAM) of computations and allows for working with larger-than-memory datasets in parallel.

In [ ]:
import dask.array as da

x = da.random.random(size=(1_000, 1_000), chunks=(250, 500))
x

In the overview notebook we looked at the task graph for the following computation:

In [ ]:
result = (x + x.T).sum(axis=0).mean()

Now let's break that down a bit and look at the task graph for just one part of that computation.

In [ ]:
x.T.visualize()

This graph demonstrates how blocked algorithms work. In the perfectly parallelizable situation, Dask can operate on each block in isolation and then reassemble the results from the outputs. Dask makes it easy to contruct graphs like this using a numpy-like API. 

## Custom Block Computations
Block computations operate on a per-block basis. So each block gets the function applied to it, and the output has the same chunk location as the input.

Some examples include the following:
- custom IO operations
- applying embarassingly parallel functions for which there is no exising Dask implementation

![map_blocks](images/custom_operations_map_blocks.png)

**Related Documentation**

   - [`dask.array.map_blocks`](https://docs.dask.org/en/latest/array-api.html?highlight=map_blocks#dask.array.Array.map_blocks)
   - [`dask.dataframe.map_partitions`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions)

### `map_blocks`

Let's imagine that there was no `da.random.random` method. We can create our own version using `map_blocks`. 

In [ ]:
import numpy as np

def random_sample():
    return np.random.random(size=(250, 500))

x = da.map_blocks(random_sample, chunks=((250, 250, 250, 250), (500, 500)), dtype=float)
x

In [ ]:
x.visualize()

> #### Understanding `chunks` argument
>
> In the example above we explicitly declare what the size of the output chunks will be ``chunks=((250, 250, 250, 250), (500, 500))`` this means 8 chunks each with shape `(250, 500)` you'll also see the chunks argument written in the short version where only the shape of one chunk is defined ``chunks=(250, 500)``. These mean the same thing.
>
> Specifying the output chunks is very useful when doing more involved operations with ``map_blocks``. By specifying ``chunks``, you can guarantee that the output will have the right shape which lets you properly chain together other operations. 
>
> When in doubt, you can always find shape and chunk information in the array representation.

In that example we created an array from scratch by passing in `dtype` and `chunks`. Next we'll consider the case of applying `map_blocks` to existing arrays.

#### Multiple arrays

``map_blocks`` can be used on single arrays or to combine several arrays. When multiple arrays are passed, ``map_blocks``
aligns blocks by block location without regard to shape.

In the following example we have two arrays with the same number of blocks
but with different shape and chunk sizes.

In [ ]:
a = da.arange(1000, chunks=(100,))
b = da.arange(100, chunks=(10,))

Let's take a look at these arrays:

In [ ]:
a

In [ ]:
b

We can pass these arrays into ``map_blocks`` using a function that takes two inputs, calculates the max of each, than then returns a numpy array of the outputs. 

In [ ]:
def func(a, b):
    return np.array([a.max(), b.max()])

result = da.map_blocks(func, a, b, chunks=(2,))
result.visualize()

#### Special arguments

There are special arguments (``block_info`` and ``block_id``) that you can use within ``map_blocks`` functions. ``block_id`` gives the index of the block within the chunks, so for a 1D array it will be something like `(i,)`. ``block_info`` is a dictionary where there is an integer key for each input dask array and a `None` key for the output array.

Let's use the example above and print ``block_info`` for the first block so that we can get a sense of what information is contained in it:

In [ ]:
from pprint import pprint

def func(a, b, block_id=None, block_info=None):
    if block_id == (0,):
        pprint(block_info)
    return np.array([a.max(), b.max()])

da.map_blocks(func, a, b, chunks=(2,)).compute()

One of the use cases for the ``block_info`` and ``block_id`` arguments is to create an array from scratch by reading in specific files for each block.

**Exercise**

Say you have a set of images that each represent a particular portion of a scene. How can you use the
technique we just learned to patch them together? 

Let's look at what is in the puzzle directory:

In [ ]:
!ls puzzle

The following cell displays the completed puzzle.

In [ ]:
from imageio import imread
import matplotlib.pyplot as plt

image = imread("puzzle/bicycle.png")
plt.imshow(image)

Now use ``map_blocks`` to read in the puzzle pieces from "bicycle_i_j.png". Note that each image piece has 3 dimensions: x, y, and RGBA.

In [ ]:
# define a function that reads one file
def reader(block, block_id=None):
    ... = block_id  # unpack block_id to get chunk location
    filename = ...  # use chunk location to get the correct file
    return imread(filename)

In [ ]:
# solution
# define a function that reads one file
def reader(block_id=None):
    ii, jj, _ = block_id                        # unpack block_id to get chunk location
    filename = f"puzzle/bicycle_{ii}_{jj}.png"  # use chunk location to get the correct file
    return imread(filename)

In [ ]:
# map that function to every block and set the expected dtype and chunk pattern of the output
result = da.map_blocks(reader, dtype=int, chunks=((24, 24), (24, 24), (4,)))
plt.imshow(result)

### ``map_partitions``

In Dask dataframe there is a similar method to ``map_blocks`` but it is called ``map_partitions``.

Here is an example of using it to check if the sum of two columns is greater than some arbitrary threshold.

In [ ]:
import dask
import dask.dataframe as dd

ddf = dask.datasets.timeseries()

result = ddf.map_partitions(lambda df, threshold: (df.x + df.y) > 0, threshold=0)
result.compute()

#### Internal uses
In practice ``map_partitions`` is used to implement many of the helper dataframe methods
that let Dask dataframe mimic Pandas. Here is the implementation of `ddf.index` for instance:

```python
@property
def index(self):
    """Return dask Index instance"""
    return self.map_partitions(
        getattr,
        "index",
        token=self._name + "-index",
        meta=self._meta.index,
        enforce_metadata=False,
    )
```

[source](https://github.com/dask/dask/blob/09862ed99a02bf3a617ac53b116f9ecf81eea338/dask/dataframe/core.py#L458-L467)


#### Understanding `meta` argument

Dask dataframes and dask arrays have a special attribute called `_meta` that allows them to know metadata about the type of dataframe/array that they represent. This metadata includes:
 - dtype (int, float)
 - column names and order
 - name
 - type (pandas dataframe, cudf dataframe)
 
**Related documentation**

- [Dataframe metadata](https://docs.dask.org/en/latest/dataframe-design.html#metadata)

This information is stored in an empty object of the proper type.

In [ ]:
print(ddf._meta)

That's how dask knows what to render when you display a dask object:

In [ ]:
print(ddf)

When you add an item to the task graph, Dask tries to run the function on the meta before you call compute. 

This approach has several benefits:

- it gives Dask a sense of what the output will look like. 
- if there are fundamental issues, Dask will fail fast

Here's a few examples. 

In [ ]:
ddf.sum()

In [ ]:
ddf.name.str.startswith("A")

See how the output looks right? The dtypes are correct, the type is a `Series` rather than a `DataFrame` like the input.

**Exercise**

Try using `startswith` on a different column and see what you get :)

In [ ]:
# solution
ddf.x.str.startswith("A")

### Declaring meta

Sometimes running the function on a miniature version of the data doesn't produce a result that is similar enough to your expected output. 

In those cases you can provide a `meta` manually.

In [ ]:
result = ddf.map_partitions(lambda df, threshold: (df.x + df.y) > threshold, threshold=0, meta=bool)
result.compute()

### `map_overlap`
Sometimes you want to operate on a per-block basis, but you need some information from neighboring blocks. 

Example operations include the following:

- Convolve a filter across an image
- Rolling sum/mean/max, …
- Search for image motifs like a Gaussian blob that might span the border of a block
- Evaluate a partial derivative

Dask Array supports these operations by creating a new array where each block is slightly expanded by the borders of its neighbors. 

![](https://docs.dask.org/en/latest/_images/overlapping-neighbors.png)

This costs an excess copy and the communication of many small chunks, but allows localized functions to evaluate in an embarrassingly parallel manner.

**Related Documentation**
   - [Array Overlap](https://docs.dask.org/en/latest/array-overlap.html)

The main API for these computations is the ``map_overlap`` method. ``map_overlap`` is very similar to ``map_blocks`` but has the additional arguments: ``depth``, ``boundary``, and ``trim``.

Here is an example of calculating the derivative:

In [ ]:
import numpy as np
import dask.array as da

a = np.array([1, 1, 2, 3, 3, 3, 2, 1, 1])
a = da.from_array(a, chunks=5)

plt.plot(a)

In [ ]:
def derivative(a):
    return a - np.roll(a, 1)

b = a.map_overlap(derivative, depth=1, boundary=None)
b.compute()

In this case each block shares 1 value from its neighboring block: ``depth``. And since we set ``boundary=0``on the outer edges of the array, the first and last block are padded with the integer 0. Since we haven't specified ``trim`` it is true by default meaning that the overlap is removed before returning the results.

If you inspect the task graph you'll see two mostly independent towers of tasks, with just some value sharing at the edges.

In [ ]:
b.visualize(collapse_outputs=True)

**Exercise**

Lets apply a gaussian filter to an image following the example from the [scipy docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html).

First create a dask array from the numpy array:

In [ ]:
from scipy import misc
import dask.array as da

a = da.from_array(misc.ascent(), chunks=(128, 128))
a

Now use ``map_overlap`` to apply ``gausian_filter`` to each block.

In [ ]:
from scipy.ndimage import gaussian_filter

b = a.map_overlap(gaussian_filter, sigma=5, ...)

In [ ]:
# solution
from scipy.ndimage import gaussian_filter

b = a.map_overlap(gaussian_filter, sigma=5, depth=10, boundary="periodic")

Check what you've come up with by plotting the results:

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.imshow(a)
ax2.imshow(b)
plt.show()

> Notice that if you set the depth to a smaller value, you can see the edges of the blocks in the output image.


## Blockwise Computations

Blockwise computations provide the infrastructure for implementing ``map_blocks``, ``reduction`` and many
of the elementwise methods that make up the Array API. 

Example operations include the following:

- simple mapping operations like adding two arrays
- operations that flip the dataset like transpose
- reductions like tensordot and inner product


<img src=images/custom_operations_map_blocks.png width=30% /><img src=images/custom_operations_blockwise.png width=30% /><img src=images/custom_operations_reduction.png align=right width=30% />

**Related Documentation**

   - [API Documentation](https://docs.dask.org/en/latest/array-api.html#dask.array.blockwise)

### Internal uses

This is the internal definition of transpose on dask.Array. In it you can see that there is a
regular ``np.transpose`` applied within each block and then the blocks are themselves transposed.

```python
def transpose(a, axes=None):
    if axes:
        if len(axes) != a.ndim:
            raise ValueError("axes don't match array")
    else:
        axes = tuple(range(a.ndim))[::-1]
    axes = tuple(d + a.ndim if d < 0 else d for d in axes)
    return blockwise(
        np.transpose, axes, a, tuple(range(a.ndim)), dtype=a.dtype, axes=axes
    )
```

[source](https://github.com/dask/dask/blob/4569b150db36af0aa9d9a8d318b4239a78e2eaec/dask/array/routines.py#L161:L170)

We can implement our own version of transpose using `blockise` directly. If you are comfortable with matrix math then the notation will look very familiar.

In [ ]:
result = da.blockwise(np.transpose, "ji", x, "ij")
result.visualize()

Using `blockwise` we can operate on multiple arrays by just passing more arguments.

In [ ]:
result = da.blockwise(np.add, "ij", x, "ij", x.T, "ij", dtype=x.dtype)
result.visualize()

This is equivalent to `x + x.T`

In [ ]:
(x + x.T).visualize()

We can increase or decrease the dimensionality of the output by changing the out index. There are lots more examples of how to use `blockwise` in the documentation.

In [ ]:
da.blockwise?

## Reduction
Each dask collection has a `reduction` method. This is the generalized method that supports operations that reduce the dimensionality of the inputs.

The difference between `blockwise` and `reduction` is that with `reduction` you have finer grained control over the behavior of the tree-reduce.

![Custom operations: reduction](images/custom_operations_reduction.png)

**Related Documentation**
   - [`dask.array.reduction`](http://dask.pydata.org/en/latest/array-api.html#dask.dataframe.Array.reduction)
   - [`dask.dataframe.reduction`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.reduction)

### Internal uses

This is the internal definition of sum on dask.Array. In it you can see that there is a
regular ``np.sum`` applied across each block and then tree-reduced with ``np.sum`` again.

```python
def sum(a, axis=None, dtype=None, keepdims=False, split_every=None, out=None):
    if dtype is None:
        dtype = getattr(np.zeros(1, dtype=a.dtype).sum(), "dtype", object)
    result = reduction(
        a,
        chunk.sum,  # this is just `np.sum`
        chunk.sum,  # this is just `np.sum`
        axis=axis,
        keepdims=keepdims,
        dtype=dtype,
        split_every=split_every,
        out=out,
    )
    return result
```
[source](https://github.com/dask/dask/blob/ac1bd05cfd40207d68f6eb8603178d7ac0ded922/dask/array/reductions.py#L344-L357)

Here is `da.sum` reimplemented as a custom reduction.

In [ ]:
da.reduction(x, np.sum, np.sum, dtype=x.dtype).visualize()

By visualizing `b` we can see how the tree reduction works. First ``sum`` is applied to each block, then every 4 chunks are combined using ``sum-partial``. This keeps going until there are less than 4 results left, then ``sum-aggregate`` is used to finish up.

**Exercise**

See how the graph changes when you set the chunks - maybe to `(100, 250)` or `(250, 250)`

In [ ]:
# solution
x = da.random.random(size=(1_000, 1_000), chunks=(100, 250))
x.sum().visualize()

### Understanding ``split_every``

``split_every`` controls the number of chunk outputs that are used as input to each ``partial`` call. 

Here is an example of doing partial aggregation on every 5 blocks along the 0 axis and every 2 blocks along the 1 axis (so 10 blocks go into each `partial-sum`)

In [ ]:
x.sum(split_every={0: 5, 1: 2}).visualize()

**Exercise**

Try setting different values for `split_every` and visualizing the task graph to see the impact.

In [ ]:
# solution
x.sum(split_every={0: 10, 1: 2}).visualize()

> **Side note**
>
> You can use reductions to calculate aggregations per-block reduction even if you don't want to combine and aggregate the results of those blocks:
>
> ```python
> da.reduction(x, np.sum, lambda x, **kwargs: x, dtype=int).compute()
> ```

### Groupby Aggregation

A different but powerful form of reduction is groupby aggregations.

There are many standard reductions supported by default on dataframe groupbys. These include methods like `mean, max, min, sum, nunique`. These are easily scaled and parallelized.

**Related Documentation**

   - [DataFrame Groupby](https://docs.dask.org/en/latest/dataframe-groupby.html#aggregate)
   - [Examples](https://examples.dask.org/dataframes/02-groupby.html)


![groupby aggregation](images/custom_operations_groupby_aggregation.png)

In order to do that you need to write three functions. These are analogous to the functions that we use for general-purpose `reduction`

- `chunk`: operates on the series groupby on each individual partition (`ddf.partitions[0].groupby("name")["x"]`)
- `agg`: operates on the concatenated output from calling chunk on every partition
- `finalize`: operates on the output from calling aggregate - returns one column. This one is actually optional.

Here's an example of a custom aggregation for calculating the mean.

In [ ]:
import dask
import dask.dataframe as dd

ddf = dask.datasets.timeseries()

custom_mean = dd.Aggregation(
    'custom_mean',
    lambda s: (s.count(), s.sum()),
    lambda count, sum: (count.sum(), sum.sum()),
    lambda count, sum: sum / count,
)
custom_result = ddf.groupby('name').agg(custom_mean)
custom_result.head()

Here is how it works:

- for every partition (one per day) group by ``name``
- on each of those pandas series groupby objects calculate the `count` and the `sum`
- concatenate every 8 (this is configurable) outputs together
- sum each of these
- finally: divide the `sum` by the `count`

This is equivalent to:

In [ ]:
simple_result = ddf.groupby('name').mean()
simple_result.head()

**NOTE**: If you look at the task graph you'll see that the structure of the computation is actually pretty different. That's because `.mean` computes the `sum` and the `count` independently and only combines the values at the end.

In [ ]:
custom_result.visualize()

In [ ]:
simple_result.visualize()

#### Why not use apply?

If you are trying to run a custom function on the groups in a groupby it can be tempting to use `.apply` but this is often a poor choice because it requires that the data be shuffled. Instead you should try writing a custom aggregation. Custom aggregation is preferable beacause, each partition is grouped (without being repartitioned) and then the results from each group on each partition are aggregated.

DON'T DO:
```python
ddf.groupby("name").apply(lambda x: x.mean())
```

This will shuffle the data so that all the data for a particular name is in the same partition. If you call `.compute()` on it you'll notice that it's much slower (about 50x on my computer).

## dask.delayed

You can wrap arbitrary functions in dask.delayed to parallelize them, but when you are operating on a Dask collection or several Dask collections, dask.delayed won't understand the organization of your blocks. You need to first can convert your collections into individual blocks set up the computation, and afterwards arrange those blocks as you like. Note that this is often the slowest approach.

Let's look back at some of our original computation and implement `da.random.random` and `transpose` using `dask.delayed`.

### Construct an array

In [ ]:
x = da.random.random(size=(1_000, 1_000), chunks=(250, 500))
x

We looked at how you can implement this using `map_blocks` at the top of this notebook. Now let's implement random using `dask.delayed`.

In [ ]:
@dask.delayed
def random_sample():
    return np.random.random(size=(250, 500))

da.concatenate(
    (da.concatenate(
        (da.from_delayed(random_sample(), shape=(250, 500), dtype=float) 
        for i in range(4)), axis=0
    ) 
    for i in range(2)), axis=1
)

### Operate on an existing array
There we saw how to create an array from scratch, now let's look at how to operate on an existing array. Let's use delayed to compute the transpose of `x`.

In [ ]:
x.T

First we'll set up the delayed function:

In [ ]:
@dask.delayed
def transpose(block):
    return np.transpose(block)

**Naive approach** - You can just pass the array right in to this function, but when you do the array is first merged into one large array and then the delayed function is applied to the whole array all at once. This can be very nice if your array isn't too large, or if your delayed function can't be coerced to work in per-block. 

Let's compare the task graphs for calling `x.T` and `transpose(x)`.

In [ ]:
x.T.visualize()

In [ ]:
transpose(x).visualize()

The better approach is to first convert the array to a nested list of delayed objects - there will be one delayed object to represent each block:

In [ ]:
delayed_array = x.to_delayed()
delayed_array

Then you can iterate over that nested list of delayed objects, convert the output to arrays and concatenate them.

In [ ]:
result = da.concatenate(
    (da.concatenate(
        (da.from_delayed(transpose(block), shape=(500, 250), dtype=float) 
        for block in row), 
        axis=0)
    for row in delayed_array),
    axis=1
)
result

In [ ]:
result.visualize()

## When to use which method

In this notebook we've covered several different mechanisms for applying arbitrary functions to the blocks in arrays or dataframes. Here's a brief summary of when you should use these various methods

- `map_block`, `map_partition` - block organization of the input matches the block organization of the output and the function is fully parallelizable. 
- `map_overlap` - block organizations of input and output match, but the function is not fully parallelizable (requires input from neighboring chunks).
- `blockwise` - same function can be applied to the blocks as to the partial and aggregated versions. Also output blocks can be in different orientations.
- `reduction` - dimensionality of output does not necessarily match that of input and function is fully parallelizable.
- `groupby().agg` - data needs to be aggregated per group (the index of the output will be the group keys).
- `dask.delayed` - data doesn't have a complex block organization or the data is small and the computation is pretty fast.